# SDXLIB Demonstration

**The intention of this notebook is to demonstrate the most basic methods available in the SDXLIB library on the most basic instance accepted by the API. Future notebooks will demonstrate the optional attributes and additional error handling.**

Once the API is in place, requests_mock and config should no longer be necessary.

In [1]:
from pprint import pprint
import requests
import requests_mock

from config import *
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

# Creating L2VPN

### Demonstration of L2VPN Client Creation and Error Handling

**This portion of the demo will create an 'SDXClient' instance using the minimum required attributes for the API request:**
* URL - "http://example.com"
* Name - Any string that is less than 50 characters.
* Endpoints - A list of dictionaries. Each dictionary must contain the following keys:
    * port_id - The Uniform Resource Name (URN) of a network device's port. Must follow the pattern: "urn:sdx:port:<oxp_url>:<node_name>:<port_name>"
    * vlan: Describes how the SDX and OXPs should treat L2VPN frames. Accepted values are:
        * "any"
        * "all"
        * "untagged"
        * integer string
        * VLAN range in the format "VLAN ID1:VLAN ID2"

In [2]:
# Define the required attributes
url = "http://example.com"
client_name = "Test L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:sax.br:Rtr01:50", "vlan": "any"},
    {"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}
]

In [3]:
# Create the L2VPN object
client = SDXClient(url, client_name, client_endpoints)

In [4]:
client

SDXClient(base_url=http://example.com, name=Test L2VPN, endpoints=[{'port_id': 'urn:sdx:port:sax.br:Rtr01:50', 'vlan': 'any'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': 'any'}])

### Error Handling

**The below mocks the call to the API which should return an error code of 409, indicating that the specified service already exists.**

In [5]:
# Create a mocker for requests for the unsuccessful case
with requests_mock.Mocker() as mock_requests:
    # Mock unsuccessful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        json=mock_response_unsuccessful_409,
        status_code=409  # Set status code for unsuccessful creation (Options are: 400, 401, 402, 409, 410, 411, 422)
    )
    
    # Debug print to verify the mock setup
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("Request successful.")
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Failed to create L2VPN. Status code: 409: L2VPN Service already exists


Request URL: http://example.com/l2vpn/1.0
L2VPN creation failed: L2VPN Service already exists


**This example will demonstrate a 'Connection error'. This should cause an SDXException.**

In [6]:
# Create a mocker for requests to simulate a RequestException
with requests_mock.Mocker() as mock_requests:
    # Mock a request exception scenario (e.g., connection error)
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0",
        exc=requests.exceptions.RequestException("Connection error")
    )
    
    # Debug print to verify the mock setup
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  
    except requests.exceptions.RequestException as e:
        print(f"Request exception occurred: {e}")  

An error occurred while creating L2VPN: Connection error


Request URL: http://example.com/l2vpn/1.0
L2VPN creation failed: An error occurred while creating L2VPN: Connection error


### Successful Call to the API

**This call should return a response from the API with the service_id attribute which has a value in the form of UUID.**

In [7]:
# Create a mocker for requests
with requests_mock.Mocker() as mock_requests:
    # Mock successful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0", 
        json=mock_response_successful,
        status_code=201  # Set status code for successful creation
    )
    
    # Debug statement to confirm URL and method
    print(f"Request URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  
        print("L2VPN creation successful!")
        pprint(response)
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  

Request URL: http://example.com/l2vpn/1.0
L2VPN creation successful!
{'service_id': '123e4567-e89b-12d3-a456-426614174000'}


# Update L2VPNs

### Demonstration of L2VPN Client Update and Error Handling

**Again, create a client instance with the minimum required attributes for a successful API call.**

The update method requires the service_id and any attributes that are to be updated. The attributes that may be updated are: name, endpoints, description, notifications, scheduling, qos_metrics, and the state attribute that is returned from the API. The state attribute may be updated to "enabled" or "disabled".

In [8]:
# Define your SDXClient instance with mock URL and initial configuration
url = "http://example.com"
client_name = "Test L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:sax.br:Rtr01:50", "vlan": "any"},
    {"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}
]

In [9]:
# Create the SDXClient instance
client = SDXClient(url, client_name, client_endpoints)

### Error Handling

**Here, we are going to attempt an update to the name, description, and state attributes. This example should demonstrate the returned exception for a 404 response, the supplied service_id is not found.**

In [10]:
# Example update_l2vpn scenario with requests_mock
with requests_mock.Mocker() as m:

    # Mock endpoint URL
    mock_url = f"{url}/l2vpn/{client.VERSION}/123e4567-e89b-12d3-a456-426614174000"
    
    # Example for handling other status codes if needed
    m.patch(mock_url, status_code=404)  # Mocking 404 response
    
    print(f"Request URL: {mock_url}")
    
    try:
        response = client.update_l2vpn(service_id="123e4567-e89b-12d3-a456-426614174000", name="New L2VPN Name", description="Updated description", state="enabled")
        print("Update Successful:")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)
    except Exception as e:
        print("Exception:", e)

Failed to update L2VPN. Status code: 404: L2VPN Service ID not found


Request URL: http://example.com/l2vpn/1.0/123e4567-e89b-12d3-a456-426614174000
SDXException: L2VPN Service ID not found


**And here I expect to see a successful response, with a description indicating that the L2VPN has been modified along with the service_id for the modified object.**

In [11]:
# Example update_l2vpn scenario with requests_mock
with requests_mock.Mocker() as m:

    # Mock endpoint URL
    mock_url = f"{url}/l2vpn/{client.VERSION}/123e4567-e89b-12d3-a456-426614174000"
    
    # Register mock response for patch request
    m.patch(mock_url, json=mock_response, status_code=200)
    
    print(f"Request URL: {mock_url}")
    
    # Call the update_l2vpn method
    try:
        response = client.update_l2vpn(service_id="123e4567-e89b-12d3-a456-426614174000", name="New L2VPN Name", description="Updated description", state="enabled")
        print("Update Successful:")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)
    except Exception as e:
        print("Exception:", e)


Request URL: http://example.com/l2vpn/1.0/123e4567-e89b-12d3-a456-426614174000
Update Successful:
{'description': 'L2VPN Service Modified',
 'service_id': '123e4567-e89b-12d3-a456-426614174000'}


# Retrieve All L2VPNs

### Demonstration to retrieve all L2VPN Clients and Error Handling

**Now we will query the API for a list of all available L2VPNs. Therefore, we will create a client that is empty except for the URL.**

There are two retrieval methods, one to retreive a list of all L2VPNs and one to retrieve the information for a single L2VPN (we will discuss this one next). 

In [12]:
client = SDXClient(url)

### Error Handling

**This error should simulate a network error.** 

In [13]:
with requests_mock.Mocker() as mock_requests:
    # Simulate a RequestException (e.g., network error)
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/",
        exc=requests.RequestException("Simulated network error")
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/")

    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns() 
        print("Get All L2VPNs (RequestException):")
        pprint(response)
    except SDXException as e:
        print(e)

ERROR:root:Failed to retrieve L2VPN(s): Simulated network error


Request URL: http://example.com/l2vpn/1.0/



**Here I expect to see an empty dictionary because there are no layer 2 VPNs.**

In [14]:
# Mock the get_all_l2vpns method
with requests_mock.Mocker() as mock_requests:
    # Mock response for no L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/",
        json=mock_response_no_l2vpns,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns()
        print("Get All L2VPNs:", response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/
Get All L2VPNs: {}


**Here I expect to see all of the active and the archived l2vpns.**

In [15]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for one or more L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/",
        json=mock_response_all_archived_l2vpns,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns()
        print("Get All L2VPNs (One or more L2VPNs exist):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/
Get All L2VPNs (One or more L2VPNs exist):
{'c73da8e1-5d03-4620-a1db-7cdf23e8978c': {'archived_date': '0',
                                          'counters_location': 'https://my.aw-sdx.net/l2vpn/7cdf23e8978c',
                                          'creation_date': '2024-05-22T00:00:00Z',
                                          'current_path': ['urn:sdx:link:tenet.ac.za:LinkToAmpath'],
                                          'description': 'Example 1',
                                          'endpoints': [{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50',
                                                         'vlan': '150'},
                                                        {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50',
                                                         'vlan': '300'}],
                                          'last_modified': '0',
                                          'name': 'VLAN between A

**Here I expect to see only the active l2vpn.**

In [16]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for one or more archived L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/0",
        json=mock_response_active_l2vpns_exist,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/0")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns(all=False, active_only=True)
        print("Get All L2VPNs (One or more L2VPNs exist):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/0
Get All L2VPNs (One or more L2VPNs exist):
{'c73da8e1-5d03-4620-a1db-7cdf23e8978c': {'archived_date': '0',
                                          'counters_location': 'https://my.aw-sdx.net/l2vpn/7cdf23e8978c',
                                          'creation_date': '20240522T00:00:00Z',
                                          'current_path': ['urn:sdx:link:tenet.ac.za:LinkToAmpath'],
                                          'description': 'Example 1',
                                          'endpoints': [{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50',
                                                         'vlan': '150'},
                                                        {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50',
                                                         'vlan': '300'}],
                                          'last_modified': '0',
                                          'name': 'VLAN between AM

**Here I expect to see only the archived l2vpns.**

In [17]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for one or more archived L2VPNs existing
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/archived",
        json=mock_response_archived_l2vpns_exist,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/archived")
    
    # Call the get_all_l2vpns method
    try:
        response = client.get_all_l2vpns(all=False, active_only=False)
        print("Get All L2VPNs (One or more L2VPNs exist):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/archived
Get All L2VPNs (One or more L2VPNs exist):
{'c73da8e1-5d03-4620-a1db-7cdf23e8978c': {'archived_date': '2024-06-16T19:20:30Z',
                                          'counters_location': 'https://my.aw-sdx.net/l2vpn/7cdf23e8978c',
                                          'creation_date': '2024-05-22T00:00:00Z',
                                          'current_path': ['urn:sdx:link:tenet.ac.za:LinkToAmpath'],
                                          'description': 'Example 1',
                                          'endpoints': [{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50',
                                                         'vlan': '150'},
                                                        {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50',
                                                         'vlan': '300'}],
                                          'last_modified': '0',
                                      

# Retrieve a Specific L2VPN

### Demonstration to retrieve a specific L2VPN Client

**Now we will query the API for a specific L2VPN. We will create a client that is empty except for the URL.**

In [18]:
client = SDXClient(url)

**I will query for the l2vpn using the service_id attribute. With this request, I should see the complete response object from the API for this l2vpn.**

In [19]:
service_id = "c73da8e1-5d03-4620-a1db-7cdf23e8978c"

with requests_mock.Mocker() as mock_requests:
    # Simulate a RequestException (e.g., network error)
    mock_requests.register_uri(
        "GET", f"{url}/l2vpn/{client.VERSION}/{service_id}",
        json=mock_response_l2vpn_exists,
        status_code=200
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    # Call the get_l2vpn method
    try:
        response = client.get_l2vpn(service_id)
        print("Get L2VPN (Successful):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
Get L2VPN (Successful):
{'c73da8e1-5d03-4620-a1db-7cdf23e8978c': {'archived_date': '0',
                                          'counters_location': 'https://my.aw-sdx.net/l2vpn/7cdf23e8978c',
                                          'creation_date': '20240522T00:00:00Z',
                                          'current_path': ['urn:sdx:link:tenet.ac.za:LinkToAmpath'],
                                          'description': 'Example 1',
                                          'endpoints': [{'port_id': 'urn:sdx:port:tenet.ac.za:Tenet03:50',
                                                         'vlan': '150'},
                                                        {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50',
                                                         'vlan': '300'}],
                                          'last_modified': '0',
                                          'name': 

# Delete an L2VPN

### Demonstration to delete an L2VPN Client and Error Handling

**We will delete an l2vpn client using the service_id attribute.**

We will begin by creating a client instance that is empty except for the URL. 

In [20]:
client = SDXClient(url)

**This will simulate a successful deletion. The return on success is None.**

In [21]:
service_id = "c73da8e1-5d03-4620-a1db-7cdf23e8978c"

with requests_mock.Mocker() as mock_requests:
    # Mock response for successful deletion
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=201
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Successful):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

Request URL: http://example.com/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
Delete L2VPN (Successful):
None


### Error Handling

**This will demonstrate the return if the service_id provided for deletion does not exist.**

In [22]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for 404: Service ID does not exist
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=404,
        json={"description": "L2VPN Service ID provided does not exist"}
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Service ID not found):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

ERROR:sdxlib.sdx_client:Failed to delete L2VPN. Status code: 404: L2VPN Service ID provided does not exist


Request URL: http://example.com/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
SDXException: L2VPN Service ID provided does not exist


**And here I expect to see a response that I don't have authorization to delete this l2vpn.**

In [23]:
with requests_mock.Mocker() as mock_requests:
    # Mock response for 401: not authorized
    mock_requests.delete(
        f"{url}/l2vpn/{client.VERSION}/{service_id}",
        status_code=401,
        json={"description": "Not Authorized"}
    )
    
    print(f"Request URL: {url}/l2vpn/{client.VERSION}/{service_id}")

    try:
        response = client.delete_l2vpn(service_id)
        print("Delete L2VPN (Not Authorized):")
        pprint(response)
    except SDXException as e:
        print("SDXException:", e)

ERROR:sdxlib.sdx_client:Failed to delete L2VPN. Status code: 401: Not Authorized


Request URL: http://example.com/l2vpn/1.0/c73da8e1-5d03-4620-a1db-7cdf23e8978c
SDXException: Not Authorized
